# Setup and Definitions

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/russellgoyder/sundial/HEAD?labpath=docs%2Fnb%2Fsundial_setup.ipynb)

We start by establishing the geometry, location and orientation of the gnomon (shadow-casting stick) and face of the sundial.

In [16]:
import sympy as sp
from sympy import sin, cos, tan
from sympy.abc import alpha, psi, sigma, i, d, iota, delta
from analemma.algebra import frame, render, util

## Fixed Stars and Earth Frames

Define an orthonormal frame (set of basis vectors) $e_1, e_2, e_3$ that is fixed relative to the "fixed stars", in [analemma.algebra.frame.base][]. Then, let the tilt of the earth's plane (axis) of rotation be $\alpha$ and measure the earth's rotation by $\psi$. The Earth frame $f_1, f_2, f_3$ is then given by [analemma.algebra.frame.planet][].

![Earth's orientation and orbit](https://raw.githubusercontent.com/russellgoyder/sundial-latex/main/figs/MainArena.png "Earth's orientation and orbit.").

In [17]:
e1, e2, e3 = frame.base("e")
f1, f2, f3 = frame.planet()
render.frame("f", (f1,f2,f3))

<IPython.core.display.Math object>

The equatorial plane should only depend on the tilt of the Earth's axis of spin $\alpha$, not the angle by which it has rotated relative to the fixed stars $\psi$.

In [18]:
render.expression(r"f_1 \wedge f_2", (f1^f2))

<IPython.core.display.Math object>

Note: the wedge operation "$\wedge$" defines a *bivector*, which encodes the orientation of a plane in space (but also has a magnitude, just as a vector has a direction and a magnitude). This is a concept from Geometric Algebra which is used throughout the `analemma` package. A good place to learn more is [https://bivector.net/](https://bivector.net/).

## Surface Frame

Define an orthonormal frame embedded in the [Earth's surface][analemma.algebra.frame.surface], with $n_1$ pointing South, $n_2$ pointing East and $n_3$ pointing up. Note that $\theta$ is not latitude, but $90^\circ$ minus latitude.

![](https://raw.githubusercontent.com/russellgoyder/sundial-latex/main/figs/SurfaceFrame.png "Frame embedded in Earth's surface.").

In [19]:
n1, n2, n3 = frame.surface()
render.frame("n", (n1,n2,n3))

<IPython.core.display.Math object>

## Orbit Rotor and Meridian Plane

Although we will work with the Earth for concreteness, the derivation applies to any planet. The angle $\sigma$ measures the progress of the planet around its orbit, with $\sigma = 0$ when the planet lies along the $e_1$ axis. Therefore, a vector from the origin to the planet models a ray of light from the star - a [sun ray][analemma.algebra.frame.sunray].

In [6]:
s = frame.sunray()
render.expression("s", s)

<IPython.core.display.Math object>

The [meridian plane][analemma.algebra.frame.meridian_plane] contains a line of longitude and is defined by $M = n_1 \wedge n_3$.

In [7]:
M = frame.meridian_plane()
render.expression("M", M)

<IPython.core.display.Math object>

The noon line is the intersection of the sun ray $s$ and the meridian plane $M$, which occurs where $s \wedge M$ (a 3-d object) vanishes.

In [20]:
render.multivector((s^M).trigsimp())

<IPython.core.display.Math object>

Solving for $\psi$ gives the angle of Earth's rotation corresponding to noon.

In [9]:
coeff = (s^M).trigsimp().get_coefs(3)[0]
soln = sp.solve(coeff.subs(sin(psi), tan(psi)*cos(psi)), tan(psi))[0]

assert soln.equals(tan(sigma)/cos(alpha))
render.expression( r"\tan(\psi)", soln )

<IPython.core.display.Math object>

## Dial Face and Gnomon

Now we can define the orientation of the sundial. The unit vector $m_3$ points upward perpendicular to the [dial][analemma.algebra.frame.dial] face, while $m_1$ and $m_2$ point North and West respectively when the face's inclination $i$ and declination $d$ are zero.

![](https://raw.githubusercontent.com/russellgoyder/sundial-latex/main/figs/DialFrame.png "Frame embedded in the sundial's face.").


In [21]:
m1, m2, m3 = frame.dial()
render.frame("m", (m1,m2,m3))

<IPython.core.display.Math object>

Given this frame, we can form the unit bivector $G_n$ that encodes the [dial face][analemma.algebra.frame.dialface] directly.

In [24]:
Gn = frame.dialface()
render.expression( "G_n", Gn)

<IPython.core.display.Math object>

The [gnomon][analemma.algebra.frame.gnomon] $g_n$ expressed relative to the planet's surface frame is given by:

![](https://raw.githubusercontent.com/russellgoyder/sundial-latex/main/figs/Gnomon.png "The gnomon.").


In [12]:
gn = frame.gnomon("n", zero_decl=False)

# extra manipulation to display exactly as in paper
render.expression("g_n", sp.collect(sp.trigsimp(gn.obj), -sin(iota)))

<IPython.core.display.Math object>

[Projected][analemma.algebra.util.project_vector] onto the fixed-stars basis, the gnomon is

In [25]:
nn1, nn2, nn3 = frame.base("n")
g = util.project_vector(gn, target_frame=(nn1, nn2, nn3), render_frame=(n1, n2, n3))
g = g.trigsimp()

render.align("g", g)

<IPython.core.display.Math object>

The gnomon lies in the meridian plane when the following trivector vanishes:

In [26]:
M_wedge_g = M^g
assert M_wedge_g.obj.trigsimp().equals((sin(delta)*sin(iota)*e1^e2^e3).obj)
render.multivector( M_wedge_g )

<IPython.core.display.Math object>

Next, we calculate the [hour angle](hour_angle.md), measured between the face of the dial and the plane containing the sun ray and the gnomon